In [50]:
from requests import get
from bs4 import BeautifulSoup

def extractor_jobs(keyword) -> list:
    """
    주어진 키워드로 We Work Remotely 웹사이트에서 채용 정보를 추출하는 함수

    Args:
        keyword (str): 검색할 채용 키워드

    Returns:
        list: 추출된 채용 정보를 담은 딕셔너리 리스트
    """
    base_url = "https://weworkremotely.com/remote-jobs/search?term=" # WWR 기본 URL
    search_term = keyword # 검색어

    response = get(f"{base_url}{search_term}") # 검색어를 포함한 URL로 GET 요청

    if response.status_code != 200: # 응답 상태 코드가 200 (성공)이 아닌 경우
        print("웹사이트에 요청할 수 없습니다.")
    else:
        results = [] # 추출된 채용 정보를 저장할 리스트
        soup = BeautifulSoup(response.text, "html.parser") # HTML 파싱
        jobs = soup.find_all("section", class_="jobs") # "jobs" 클래스를 가진 모든 섹션 검색 (채용 정보 섹션)

        for job_section in jobs: # 각 채용 정보 섹션 순회
            job_posts = job_section.find_all('li') # 각 섹션 내의 모든 채용 게시글 검색
            job_posts.pop(-1) # 마지막 li 요소 제거 (보통 "더 많은 채용 정보" 링크)

            for post in job_posts: # 각 채용 게시글 순회
                anchors = post.find_all('a') # 모든 <a> 태그 검색
                anchor = anchors[1] # 두 번째 <a> 태그 선택 (채용 링크)
                link = anchor['href'] # 채용 링크 추출
                company_spans = anchor.find_all('span', class_='company') # 회사 정보를 담은 모든 <span> 태그 검색

                if len(company_spans) >= 2:  # 회사 및 종류가 있는지 확인
                    company = company_spans[0].string # 회사명 추출
                    kind = company_spans[1].string # 채용 종류 추출
                    region = company_spans[2].string if len(company_spans) >= 3 else "N/A" # 지역 정보 추출, 없으면 "N/A"
                elif len(company_spans) == 1: # 회사만 있는 경우
                    company = company_spans[0].string
                    kind = "N/A"
                    region = "N/A"
                else: # 회사 및 종류가 모두 없는 경우
                    company = "N/A"
                    kind = "N/A"
                    region = "N/A"
                    continue # 이 채용 공고는 건너뜀

                title = anchor.find('span', class_='title') # 채용 제목 추출
                job_data = { # 채용 정보 딕셔너리 생성
                    'link': link,
                    'company': company,
                    'region': region,
                    'position': title.string
                }
                results.append(job_data) # 채용 정보 리스트에 추가

        return results # 추출된 채용 정보 리스트 반환

In [ ]:
from requests import get
from bs4 import BeautifulSoup

def extractor_jobs(keyword) -> list:
    base_url = "https://weworkremotely.com/remote-jobs/search?term="
    search_term = keyword
    response = get(f"{base_url}{search_term}")

    if response.status_code != 200:
        print("웹사이트에 요청할 수 없습니다.")
        return []

    results = []
    soup = BeautifulSoup(response.text, "html.parser")
    jobs = soup.find_all("section", class_="jobs")

    for job_section in jobs:
        job_posts = job_section.find_all('li')
        if job_posts:
            job_posts.pop(-1)

        for post in job_posts:
            anchor = post.find('a', href=True)
            if not anchor:
                continue

            # 웹 페이지 수정으로 인한 구조 변경
            link = "https://weworkremotely.com" + anchor['href']

            # 회사 정보 추출
            company_tag = post.find('p', class_='new-listing__company-name')
            company = company_tag.get_text(strip=True) if company_tag else "N/A"

            # 지역 정보 추출
            region_tag = post.find('p', class_='new-listing__company-headquarters')
            region = region_tag.get_text(strip=True) if region_tag else "N/A"

            # 채용 포지션(제목) 추출
            title_tag = post.find('h4', class_='new-listing__header__title')
            position = title_tag.get_text(strip=True) if title_tag else "N/A"

            job_data = {
                'link': link,
                'company': company,
                'region': region,
                'position': position
            }
            results.append(job_data)

    return results


In [52]:
extractor_jobs("python")

[{'link': 'https://weworkremotely.comhttps://metana.io/full-stack-software-engineer-bootcamp/?utm_source=weworkremotely.com&utm_medium=homepage-ad',
  'company': 'Metana',
  'region': 'No coding experience required, Land a job with 1-on-1 expert support.',
  'position': 'Coding Tech Bootcamp - Job Guaranteed by Metana'},
 {'link': 'https://weworkremotely.com/company/lifeanalytics',
  'company': 'LifeAnalytics',
  'region': 'japan',
  'position': 'React&Python,R :Bio&Pathology Field Senior Full Stuck Developer'},
 {'link': 'https://weworkremotely.com/company/doximity',
  'company': 'Doximity',
  'region': 'San Francisco, CA',
  'position': 'Software Engineer (Python), Data Platform'},
 {'link': 'https://weworkremotely.com/company/speechify-inc',
  'company': 'Speechify Inc',
  'region': 'Florida',
  'position': 'Senior Software Engineer, ML Platform & Data Acquisition (Python)'},
 {'link': 'https://weworkremotely.com/company/soda-data',
  'company': 'Soda Data',
  'region': 'Brussels Be